# Procesamiento en tiempo real (Apche Spark Streaming & Apche Kafka)

### Eliminar Carpetas versiones kafka

In [ ]:
# Eliminar Carpetas versiones kafka
#!rm -rf /content/kafka_2.12-3.9.0
!rm -rf /content/kafka_2.13-3.9.0.tgz

#!rm -rf /content/kafka_2.13-4.0.0
!rm -rf /content/kafka_2.13-4.0.0.tgz

## Procesamiento en tiempo real (Spark Streaming & Kafka):  

1.   Configurar un topic en Kafka para simular la llegada de datos en tiempo real (usar un generador de datos).
2.   Implementar una aplicación Spark Streaming que consuma datos del topic de Kafka.
3. Realizar algún tipo de procesamiento o análisis sobre los datos en tiempo real (contar eventos, calcular estadísticas, etc.).
4. Visualizar los resultados del procesamiento en tiempo real.

## Configurar Kafka en Colab

### Instalar Java y Kafka

In [ ]:
# Instalar Java 8 (requerido por Kafka)
!apt-get update && apt-get install -y openjdk-8-jdk

# Descargar Kafka 4.0.0
!wget https://dlcdn.apache.org/kafka/3.9.0/kafka_2.12-3.9.0.tgz
!tar -xzf kafka_2.12-3.9.0.tgz

# Configurar variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["KAFKA_HOME"] = "/content/kafka_2.12-3.9.0"

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,788 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,685 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,5

## Iniciar Servicios de Kafka
### Iniciar Zookeeper y Kafka

In [ ]:
# Iniciar Zookeeper (en segundo plano)
!$KAFKA_HOME/bin/zookeeper-server-start.sh -daemon $KAFKA_HOME/config/zookeeper.properties

# Iniciar Kafka Server (en segundo plano)
!$KAFKA_HOME/bin/kafka-server-start.sh -daemon $KAFKA_HOME/config/server.properties

# Esperar 10 segundos para que los servicios inicien
import time
print("Esperando 10 segundos para que los servicios inicien...")
time.sleep(10)

## Variables de entorno globales
!echo $KAFKA_HOME
!echo $JAVA_HOME

Esperando 10 segundos para que los servicios inicien...
/content/kafka_2.12-3.9.0
/usr/lib/jvm/java-8-openjdk-amd64


In [ ]:
# Confirmar instalacion de paquetes reuqridos
# !pip install kafka-python
# !python3 -m pip list | grep kafka

!ps -ef | grep kafka
!ps -ef | grep spark
!ps -ef | grep java

#!apt list --installed | grep java
!which java
#!pip list | grep kafka

root        2911     504  0 12:18 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root        2913    2911  0 12:18 ?        00:00:00 grep kafka
root        2914     504  0 12:18 ?        00:00:00 /bin/bash -c ps -ef | grep spark
root        2916    2914  0 12:18 ?        00:00:00 grep spark
root        2258       1  4 12:17 ?        00:00:01 /usr/lib/jvm/java-8-openjdk-amd64/bin/java -Xmx5
root        2667       1 19 12:17 ?        00:00:07 /usr/lib/jvm/java-8-openjdk-amd64/bin/java -Xmx1
root        2917     504  0 12:18 ?        00:00:00 /bin/bash -c ps -ef | grep java
root        2919    2917  0 12:18 ?        00:00:00 grep java
/usr/bin/java


## Verificación de puertos

In [ ]:
# Verificación de puertos

print("Zookeeper y Kafka Broker están escuchando en los puertos esperados.")
# Verificar Zookeeper
!netstat -tuln | grep 2181
# Verificar Kafka Broker
!netstat -tuln | grep 9092

Zookeeper y Kafka Broker están escuchando en los puertos esperados.
tcp        0      0 0.0.0.0:2181            0.0.0.0:*               LISTEN     
tcp        0      0 0.0.0.0:9092            0.0.0.0:*               LISTEN     


## Crear Topic en Kafka
### Crear Topic para Salarios

In [ ]:
#  eliminar un topic en Apache Kafka
#!$KAFKA_HOME/bin/kafka-topics.sh --delete --bootstrap-server localhost:9092 --topic test-topic
!$KAFKA_HOME/bin/kafka-topics.sh --delete --bootstrap-server localhost:9092 --topic salaries-stream


# Crear topic "salaries-stream" con 1 partición
!$KAFKA_HOME/bin/kafka-topics.sh --create \
--bootstrap-server localhost:9092 \
--replication-factor 1 \
--partitions 1 \
--topic salaries-stream

# Crear topic "salaries_stream" con 3 particiónes y dos replicar o broker
#!$KAFKA_HOME/bin/kafka-topics.sh --create \
#--bootstrap-server localhost:9092 \
#--replication-factor 2 \
#--partitions 3 \
#--topic salaries-stream

Created topic salaries-stream.


## Verificar topic creado

In [ ]:
# Verificar topic creado
!$KAFKA_HOME/bin/kafka-topics.sh --list --bootstrap-server localhost:9092

salaries-stream


## Verificar Topics -> Veriones antiguas

In [ ]:
# Verificar Topics
# Veriones antigua
#!$KAFKA_HOME/bin/kafka-topics.sh --list --zookeeper localhost:2181

## Verificar Detalles del Topic -> Veriones antigua

In [ ]:
# Verificar Detalles del Topic
# Veriones antigua
# !$KAFKA_HOME/bin/kafka-topics.sh --describe --zookeeper localhost:2181 --topic salaries_stream

## Verificar Logs

### Verificar Logs -> zookeeper

In [ ]:
# Verificar Logs
!cat $KAFKA_HOME/logs/zookeeper.out

[2025-04-01 12:17:56,446] INFO Reading configuration from: /content/kafka_2.12-3.9.0/config/zookeeper.properties (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO clientPortAddress is 0.0.0.0:2181 (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO secureClientPort is not set (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO observerMasterPort is not set (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO metricsProvider.className is org.apache.zookeeper.metrics.impl.DefaultMetricsProvider (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,458] INFO autopurge.snapRetainCount set to 3 (org.apache.zookeeper.server.DatadirCleanupManager)
[2025-04-01 12:17:56,459] INFO autopurge.purgeInterval set to 0 (org.apache.zookeeper.server.DatadirCleanupManager)
[2025-04-01 12:17:56,459] INFO Purge task is not scheduled. (org.apache.zookeeper

### Verificar Logs -> server

In [ ]:
# Verificar Logs
!cat $KAFKA_HOME/logs/server.log

[2025-04-01 12:17:56,446] INFO Reading configuration from: /content/kafka_2.12-3.9.0/config/zookeeper.properties (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO clientPortAddress is 0.0.0.0:2181 (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO secureClientPort is not set (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO observerMasterPort is not set (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,455] INFO metricsProvider.className is org.apache.zookeeper.metrics.impl.DefaultMetricsProvider (org.apache.zookeeper.server.quorum.QuorumPeerConfig)
[2025-04-01 12:17:56,458] INFO autopurge.snapRetainCount set to 3 (org.apache.zookeeper.server.DatadirCleanupManager)
[2025-04-01 12:17:56,459] INFO autopurge.purgeInterval set to 0 (org.apache.zookeeper.server.DatadirCleanupManager)
[2025-04-01 12:17:56,459] INFO Purge task is not scheduled. (org.apache.zookeeper

## Validar ingesta de datos: Verificar que el generador envía datos al topic.

In [ ]:
# En otra celda, inicia un consumer para recibir los mensajes:
## Validar ingesta de datos: Verificar que el generador envía datos al topic.
!$KAFKA_HOME/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic salaries-stream --from-beginning

Processed a total of 0 messages


## Describe el tema para obtener detalles sobre la configuración.

In [ ]:
!$KAFKA_HOME/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic salaries-stream
#!$KAFKA_HOME/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic test_topic

Topic: salaries-stream	TopicId: yCekz4WuRqm-CoZZa6F9bw	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: salaries-stream	Partition: 0	Leader: 0	Replicas: 0	Isr: 0	Elr: N/A	LastKnownElr: N/A


## Generador de Datos en Tiempo Real
### Script para Generar Datos Fake

In [ ]:
# Confirmar instalacion de paquetes reuqridos
!pip install kafka-python

import json
import random
from time import sleep
import pandas as pd
from kafka import KafkaProducer

# Configurar producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

# Listas de datos fake
jobs = ["Data Scientist", "ML Engineer", "Data Analyst", "BI Developer"]
countries = ["US", "IN", "GB", "CA", "DE", ]

def generate_fake_data():
    while True:
        data = {
            "timestamp": str(pd.Timestamp.now()),
            "job_title": random.choice(jobs),
            "salary_usd": random.randint(30000, 150000),
            "country": random.choice(countries)
        }
        producer.send('salaries-stream', value=data)
        sleep(random.uniform(0.1, 0.5))  # Simular flujo continuo
        print(f"Mensaje enviado: {data}")

# Ejecutar en segundo plano
import threading
thread = threading.Thread(target=generate_fake_data)
thread.start()

print("Generador de datos iniciado. Enviando mensajes a Kafka...")

Mensaje enviado: {'timestamp': '2025-04-01 12:38:22.441987', 'job_title': 'ML Engineer', 'salary_usd': 44674, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:22.768861', 'job_title': 'Data Scientist', 'salary_usd': 130371, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:22.997732', 'job_title': 'ML Engineer', 'salary_usd': 34665, 'country': 'IN'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:22.922779', 'job_title': 'Data Scientist', 'salary_usd': 107962, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:23.127923', 'job_title': 'Data Scientist', 'salary_usd': 124720, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:23.135004', 'job_title': 'ML Engineer', 'salary_usd': 136350, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:23.273276', 'job_title': 'ML Engineer', 'salary_usd': 148586, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:23.440922', 'job_title': 'BI Developer', 'salary

## Aplicación Spark Streaming
### Configurar Spark para Streaming

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Crear sesión Spark con soporte para Streaming
spark = SparkSession.builder \
    .appName("RealTimeSalaries") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1") \
    .config("spark.streaming.kafka.consumer.maxPollRecords", "100") \
    .config("spark.streaming.kafka.consumer.maxPollIntervalMs", "1000") \
    .getOrCreate()

# Leer stream de Kafka
stream_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "salaries-stream") \
    .option("startingOffsets", "latest") \
    .option("checkpointLocation", "/path/to/checkpoint") \
    .load()

Mensaje enviado: {'timestamp': '2025-04-01 12:38:46.062278', 'job_title': 'BI Developer', 'salary_usd': 79652, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:46.148623', 'job_title': 'Data Analyst', 'salary_usd': 80392, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:46.007604', 'job_title': 'BI Developer', 'salary_usd': 97390, 'country': 'US'}


## Procesamiento en Tiempo Real
## Transformaciones de Streaming

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

# Convertir JSON a DataFrame estructurado
schema = StructType([
    StructField("timestamp", TimestampType()),
    StructField("job_title", StringType()),
    StructField("salary_usd", IntegerType()),
    StructField("country", StringType())
])

parsed_df = stream_df \
    .select(from_json(col("value").cast("string"), schema).alias("data")) \
    .select("data.*")

# Filtrado de datos corruptos:
parsed_df = parsed_df.filter(col("salary_usd").isNotNull())

# Calcular métricas cada 10 segundos
# Watermarking: Para manejar datos tardíos.
windowed_counts = parsed_df \
    .withWatermark("timestamp", "1 minute") \
    .groupBy(
        window("timestamp", "10 seconds"),
        "job_title"
    ) \
    .agg(
        count("*").alias("total_positions"),
        avg("salary_usd").alias("avg_salary")
    )

Mensaje enviado: {'timestamp': '2025-04-01 12:38:54.380287', 'job_title': 'ML Engineer', 'salary_usd': 95427, 'country': 'CA'}
Mensaje enviado: {'timestamp': '2025-04-01 12:38:54.506726', 'job_title': 'BI Developer', 'salary_usd': 134658, 'country': 'US'}


## Visualización en Tiempo Real
### Mostrar Resultados en Consola

In [ ]:
print("Streaming iniciado. Esperando datos...")

# Configurar query de escritura
query = windowed_counts.writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", False) \
    .trigger(processingTime="10 seconds") \
    .start()

# Mantener el stream activo por 1 minuto (para pruebas) -> 60 seg
query.awaitTermination(60)

Mensaje enviado: {'timestamp': '2025-04-01 12:39:03.959708', 'job_title': 'BI Developer', 'salary_usd': 121199, 'country': 'DE'}
Streaming iniciado. Esperando datos...
Presiona Ctrl+C para detener el streaming.
Mensaje enviado: {'timestamp': '2025-04-01 12:39:04.181476', 'job_title': 'ML Engineer', 'salary_usd': 86348, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:04.155004', 'job_title': 'BI Developer', 'salary_usd': 99591, 'country': 'CA'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:04.000537', 'job_title': 'Data Analyst', 'salary_usd': 144489, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:04.382802', 'job_title': 'Data Analyst', 'salary_usd': 123289, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:04.500223', 'job_title': 'Data Scientist', 'salary_usd': 31541, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:04.439528', 'job_title': 'Data Analyst', 'salary_usd': 132714, 'country': 'US'}
Mensaje enviado: 

False

### Dashboard interactivo:

In [ ]:
# Ejemplo con matplotlib:
import matplotlib.pyplot as plt
from IPython import display

def update_plot(df, epoch_id):
    plt.clf()
    df.plot(kind='bar', x='job_title', y='avg_salary')
    display.display(plt.gcf())
    display.clear_output(wait=True)

query = windowed_counts.writeStream \
    .foreachBatch(update_plot) \
    .start()

Mensaje enviado: {'timestamp': '2025-04-01 12:39:20.928276', 'job_title': 'Data Analyst', 'salary_usd': 40919, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:21.023968', 'job_title': 'BI Developer', 'salary_usd': 83693, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:39:21.077982', 'job_title': 'Data Scientist', 'salary_usd': 32161, 'country': 'US'}


## Verificar con Kafka Console Producer y Consumer

In [ ]:
# Verificar con Kafka Console Producer y Consumer
!$KAFKA_HOME/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic salaries-stream

Mensaje enviado: {'timestamp': '2025-04-01 12:40:26.807302', 'job_title': 'Data Scientist', 'salary_usd': 79521, 'country': 'IN'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:26.753502', 'job_title': 'Data Scientist', 'salary_usd': 37242, 'country': 'CA'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:26.797111', 'job_title': 'ML Engineer', 'salary_usd': 99077, 'country': 'IN'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:26.949096', 'job_title': 'Data Analyst', 'salary_usd': 65918, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:26.971411', 'job_title': 'BI Developer', 'salary_usd': 142410, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:27.061126', 'job_title': 'ML Engineer', 'salary_usd': 142332, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:27.210955', 'job_title': 'Data Analyst', 'salary_usd': 76314, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:40:27.089581', 'job_title': 'Data Analyst', 'salary_us

## Validar ingesta de datos: Verificar que el generador envía datos al topic.

In [ ]:
# En otra celda, inicia un consumer para recibir los mensajes:
!$KAFKA_HOME/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic salaries-stream --from-beginning

Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.528263', 'job_title': 'ML Engineer', 'salary_usd': 48121, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.442431', 'job_title': 'ML Engineer', 'salary_usd': 61892, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.654424', 'job_title': 'Data Analyst', 'salary_usd': 52291, 'country': 'US'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.759058', 'job_title': 'ML Engineer', 'salary_usd': 127677, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.816741', 'job_title': 'Data Scientist', 'salary_usd': 38399, 'country': 'IN'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.883484', 'job_title': 'ML Engineer', 'salary_usd': 36656, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:32.866117', 'job_title': 'Data Scientist', 'salary_usd': 118080, 'country': 'GB'}
Mensaje enviado: {'timestamp': '2025-04-01 12:41:33.029357', 'job_title': 'ML Engineer', 'salary_usd':

## Detener Servicios (Celda Final)

In [ ]:
# Detener producer y servicios
producer.close()
print("Producer detenido.")
!$KAFKA_HOME/bin/kafka-server-stop.sh
!$KAFKA_HOME/bin/zookeeper-server-stop.sh
print("Servicios detenidos.")

# Parada Controlada:
query.stop()
print("Streaming detenido")
spark.stop()
print("Spark Streaming detenido.")

Exception in thread Thread-9 (generate_fake_data):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-35-d2c5696e5fe3>", line 28, in generate_fake_data
  File "/usr/local/lib/python3.11/dist-packages/kafka/producer/kafka.py", line 601, in send
    assert not self._closed, 'KafkaProducer already closed!'
           ^^^^^^^^^^^^^^^^
AssertionError: KafkaProducer already closed!
Exception in thread Thread-22 (generate_fake_data):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-46-106ac6026271>", line 28, in generate_fake_data
  File "/usr/local/lib/python3.11/dist-packages/kafka/producer/k

Producer detenido.
Mensaje enviado: {'timestamp': '2025-04-01 12:42:29.742439', 'job_title': 'BI Developer', 'salary_usd': 111917, 'country': 'DE'}
Mensaje enviado: {'timestamp': '2025-04-01 12:42:29.606201', 'job_title': 'Data Analyst', 'salary_usd': 46535, 'country': 'GB'}
No kafka server to stop
No zookeeper server to stop


Exception in thread Thread-20 (generate_fake_data):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-45-fd934becb730>", line 28, in generate_fake_data
  File "/usr/local/lib/python3.11/dist-packages/kafka/producer/kafka.py", line 601, in send
    assert not self._closed, 'KafkaProducer already closed!'
           ^^^^^^^^^^^^^^^^
AssertionError: KafkaProducer already closed!


Servicios detenidos.
Streaming detenido
Mensaje enviado: {'timestamp': '2025-04-01 12:42:29.717025', 'job_title': 'Data Scientist', 'salary_usd': 149752, 'country': 'CA'}
Spark Streaming detenido.
